# 作业二

    对比单隐层sigmoid激活函数的神经网络与逻辑回归效果的区别，尝试拟合隐层2、3、4、5、10个神经元的区别

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.utils import to_categorical

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
data=pd.read_csv('./data/glass.csv')
print(data.head())
x_input=np.array(data.iloc[:,:-1])
y=np.array(data['Type']).reshape(-1,1)
y_input=to_categorical(y-1)

        RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  Type
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0     1
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0     1
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0     1
3  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0     1
4  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0     1


##  2节点误差
    训练网络1000次，输出每50步误差，2节点网络训练1000次后，误差为1.5169532

In [3]:
#定义层
def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights=tf.Variable(tf.random_normal((in_size, out_size)))
    biases=tf.Variable(tf.zeros((1, out_size)) + 0.1)
    Wx_plus_b=tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs=Wx_plus_b
    else:
        outputs=activation_function(Wx_plus_b)
    return outputs
#构建单隐层网络
#传入数据
xs=tf.placeholder(tf.float32,[None,x_input.shape[-1]])
ys=tf.placeholder(tf.float32,[None,y_input.shape[-1]])
#隐层1
l1=add_layer(xs,x_input.shape[-1],2,activation_function=tf.nn.sigmoid)
#输出层
prediction=add_layer(l1,2,y_input.shape[-1],activation_function=tf.nn.softmax)
#定义损失
cross_entropy=tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
#定义何如优化损失
train=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)
#初始化变量
init=tf.global_variables_initializer()

#开启会话
with tf.Session() as sess:  
    sess.run(init)
    for i in range(1000):
        sess.run(train, feed_dict={xs: x_input, ys: y_input})
        if i%50==0:
            print(sess.run(cross_entropy,feed_dict={xs:x_input,ys:y_input}))

1.933957
1.6349906
1.5698303
1.5494598
1.5400926
1.5346065
1.5309275
1.5282497
1.5261918
1.5245492
1.5232006
1.5220699
1.5211065
1.5202739
1.519547
1.5189058
1.5183362
1.5178269
1.5173682
1.5169532


## 3节点
    训练网络1000次，输出每50步误差，3节点训练误差1.5126112

In [4]:
def train_units(n):
    l1=add_layer(xs,x_input.shape[-1],n,activation_function=tf.nn.sigmoid)
    prediction=add_layer(l1,n,y_input.shape[-1],activation_function=tf.nn.softmax)
    with tf.Session() as sess:
        sess.run(init)
        for i in range(1000):
            sess.run(train, feed_dict={xs: x_input, ys: y_input})
            if i%50==0:
                print(sess.run(cross_entropy,feed_dict={xs:x_input,ys:y_input}))
train_units(3)

1.8141459
1.549244
1.5294211
1.5240554
1.5214074
1.5196786
1.518405
1.5174098
1.5166043
1.5159366
1.5153724
1.5148897
1.5144717
1.5141057
1.513783
1.5134963
1.5132402
1.5130094
1.5128007
1.5126112


## 4节点误差
    1000次训练误差为1.512973

In [5]:
train_units(4)

2.1956966
1.582459
1.5407168
1.5303158
1.5256819
1.5228505
1.52086
1.5193639
1.5181937
1.5172533
1.5164803
1.5158347
1.5152863
1.5148158
1.5144073
1.5140492
1.513733
1.5134518
1.5131998
1.512973


## 五节点误差
    1000次训练误差1.5130552

In [6]:
train_units(5)

2.1113255
1.5870216
1.5470468
1.5337846
1.5274926
1.5239204
1.5215795
1.5198933
1.5186055
1.5175847
1.5167537
1.5160638
1.5154816
1.514984
1.5145534
1.5141778
1.5138469
1.5135533
1.513291
1.5130552


## 10节点误差
    1000次网络训练误差为1.5128496

In [7]:
train_units(10)

2.4642775
1.5914952
1.5437355
1.5308102
1.5250441
1.5219922
1.5200658
1.51869
1.5176338
1.5167868
1.5160885
1.515502
1.5150008
1.5145682
1.5141907
1.5138583
1.5135636
1.5133
1.5130634
1.5128496


## 总结
    因为神经网络参数一开始是随机设置的，每次运行结果都会不一样，但最终每个网络的误差均在1.51左右

# 逻辑回归

In [8]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [9]:
X=data.iloc[:,:-1]
Y=data['Type'].values-1
Y=Y.reshape(-1,1)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

In [10]:
lr=LogisticRegression()
lr.fit(X_train_std,Y_train)
pred=lr.predict(X_test_std)
print('训练结果：\n',pred)

print('错分率：\n',sum(pred.reshape(-1,1)!=Y_test)/len(Y_test))


训练结果：
 [6 0 1 1 1 1 0 1 1 1 1 0 1 1 1 6 0 0 0 1 4 0 6 6 0 0 6 0 1 0 0 0 1 0 0 0 0
 0 0 6 4 5 1 0 0 1 0 1 1 1 0 5 6 0 1 1 0 1 1 0 0 1 0 1 0]
错分率：
 [0.43076923]


C:\jupy\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [60]:
#交叉熵损失
aa=pred.reshape(-1,1)!=Y_test
ab=np.where(aa,1,0)
cc=[i for i,x in enumerate(ab) if x==0]
loss=0
correct_s=to_categorical(pred[cc]-1)
correct_p=lr.predict_log_proba(X_test_std)[cc]
for i in range(len(cc)):
    loss+=np.dot(correct_s[i],correct_p[i])
print('交叉熵损失：\n',-loss/len(Y_test))

交叉熵损失：
 1.5721781934638335


### 多分类逻辑回归交叉熵损失为1.57217，比神经网络的1.51大